In [9]:
import pandas as pd
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [10]:
backtesting.lib

<module 'backtesting.lib' from '/Users/jacktheripper/anaconda3/lib/python3.8/site-packages/backtesting/lib.py'>

In [93]:
data = pd.read_csv('ETHBTC-1d-data.csv')
data.set_index('timestamp')

In [116]:
data.index

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08',
               '2018-01-09', '2018-01-10',
               ...
               '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31',
               '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
               '2022-01-05', '2022-01-06'],
              dtype='datetime64[ns]', name='timestamp', length=1467, freq=None)

In [117]:
data = data[['open','high','low','close']]
data.rename(columns = {'open' : 'Open', 'high' : 'High', 'low' : 'Low', 'close' : 'Close'}, inplace = True)

/Users/jacktheripper/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [118]:
def SMA(values, n):
    return pd.Series(values).rolling(n).mean()

def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return pd.Series(ema)

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 40
    n2 = 80
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [119]:
bt = Backtest(data, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2018-01-01 00:00:00
End                       2022-01-06 00:00:00
Duration                   1466 days 00:00:00
Exposure Time [%]                   90.933879
Equity Final [$]                  8204.775746
Equity Peak [$]                  21975.342097
Return [%]                         -17.952243
Buy & Hold Return [%]                  40.545
Return (Ann.) [%]                   -4.803893
Volatility (Ann.) [%]               50.499226
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -69.247264
Avg. Drawdown [%]                  -10.807837
Max. Drawdown Duration      376 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                   23
Win Rate [%]                        47.826087
Best Trade [%]                      47.263393
Worst Trade [%]                    -43.136031
Avg. Trade [%]                    

In [21]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

/Users/jacktheripper/anaconda3/lib/python3.8/site-packages/backtesting/backtesting.py:1361: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 2.99 s, sys: 31.9 ms, total: 3.02 s
Wall time: 3.08 s


Start                                     0.0
End                                    1466.0
Duration                               1466.0
Exposure Time [%]                   97.068848
Equity Final [$]                120383.935759
Equity Peak [$]                 178304.590784
Return [%]                        1103.839358
Buy & Hold Return [%]                  40.545
Return (Ann.) [%]                         NaN
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                  -47.893307
Avg. Drawdown [%]                   -6.515129
Max. Drawdown Duration                  258.0
Avg. Drawdown Duration                   22.1
# Trades                                 63.0
Win Rate [%]                        52.380952
Best Trade [%]                        94.6094
Worst Trade [%]                    -18.994996
Avg. Trade [%]                    

In [22]:
stats._strategy

<Strategy SmaCross(n1=10,n2=20)>

In [120]:
bt.plot(plot_volume=False, plot_pl=False)

In [62]:
lm = LinearRegression()

In [64]:
lm.fit(X_train, y_train)

LinearRegression()

In [65]:
print(lm.intercept_)

[-3.09402576e-05]


In [66]:
lm.coef_

array([[-0.43183242,  0.71535947,  0.71826062]])

In [67]:
X.columns

Index(['Open', 'High', 'Low'], dtype='object')